# 2강. 파이썬 응용 및 크롤링

데이터를 분석하기에 앞서 직접 데이터를 수집합니다. AI 모델링의 시작은 데이터에 대한 이해를 높이는 것입니다. 최근 Data-driven Modeling 방식이 대두되고 있습니다. 모델일 상향 평준화된 만큼, 모델 구조를 바꾸기보다는 어떤 데이터를 입력해야 성능을 높일 수 있는지 연구하는 것입니다. 아무리 모델이 좋더라도 학습에 활용되는 데이터의 퀄리티가 높지 않으면 성능이 떨어질 수밖에 없습니다. 또한, 모델링에 집중하다 보면 내가 처리하고자 하는 데이터에 대한 이해를 놓치는 경우가 많습니다. 비록 앞으로 직접 데이터를 수집하는 일이 많지 않다고 하더라도, 데이터를 먼저 살펴보시기를 권장합니다.

## Selenium 활용 동적 크롤링

Selenium은 웹을 동작시키는 하나의 도구입니다. Colab 환경에서는 실제로 웹이 동작하는 화면을 띄울 수는 없지만, 가상의 브라우저를 동작시킬 수 있습니다.

Selenium 패키지는 자주 업데이트 되기 때문에 Colab 환경에서 Selenium 설치하는 방법은 계속 매번 달라집니다. 여기서 코드는 [블로그](https://velog.io/@kite_day/colab-%EC%97%90%EC%84%9C-%EC%9B%B9-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0-selenium)를 참고했습니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' # (최초 1회)
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [908 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit

In [3]:
# selenium 설치 확인
!python --version

import selenium
print(selenium.__version__)

Python 3.10.12
4.31.1


In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import chromedriver_autoinstaller  # setup chrome options

In [5]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

In [6]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Colab은 새창을 지원하지않기 때문에 창을 띄우지 않는 Headless 모드로 실행해야 합니다.
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

## 파리 2024 올림픽 네이버 뉴스 기사 데이터 수집

Selenium과 BeautifulSoup을 활용해서 네이버에서 제공하는 파리 2024 올림픽 뉴스 기사를 수집합니다.

In [75]:
driver = webdriver.Chrome(options=chrome_options)

In [47]:
# 네이버에서 제공하는 PARIS 2024 링크
url = 'https://m.sports.naver.com/paris2024/news?date=20240803&sort=popular&isPhoto=N' # URL을 자세히 살펴보면 우리가 검색한 정보를 찾아볼 수 있습니다.

# 드라이버로 URL 접속하기
driver.get(url)

In [48]:
# 드라이버에서 페이지 소스 코드 불러오기
html = driver.page_source

In [49]:
# BeautifulSoup 객체 생성
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [52]:
# 정규표현식 라이브러리
import re

# BeautifulSoup으로 뉴스 리스트 영역 설정
news_section = soup.find('div', re.compile('NewsList_comp_news_list'))

# 영역 내에서 뉴스 리스트 불러오기
news_tag_list = news_section.find_all('li', re.compile('NewsList'))
news_tag_list

[<li class="NewsList_list_item__gQUBA"><a class="NewsList_link_news__pZ5s8" href="/paris2024/article/011/0004375451"><div class="NewsList_image_area__95MoG"><img alt="" class="NewsList_image__ucYpU" src="https://dthumb-phinf.pstatic.net/?src=%22https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Fthumb154%2F011%2F2024%2F08%2F03%2F4375451.jpg%3Ftype%3Dnf440_300%22&amp;type=nf440_300&amp;service=sports"/></div><div class="NewsList_info_area__TTF0X"><span class="NewsList_title__DgFuU">"상대 선수가 선 넘었다" 은메달 따고 눈물 '펑펑' 김민종에 쏟아진 위로·응원</span><span class="NewsList_sub_text__7irLO">2일(현지시간) 프랑스 파리 아레나 샹드마르스에서 열린 2024 파리올림픽 유도 남자 100kg 이상급 시상식에서 은메달을 획득한 김민종이 금메달을 획득한 프랑스 테디 리네르와 기념촬영을 하고 있다. 연합뉴스[서울경제] 한국 유도 역사상 최초의 올림픽 최중량급 은메달을 딴 김민종(23·양평군청)은 하늘도 감동해 금메달을...</span><div class="NewsList_sub_area__K2+bN"><span class="NewsList_press__D0KZ7">서울경제</span><span class="NewsList_data__WO0hy">조회수 363,715</span></div></div></a></li>,
 <li class="NewsList_list_item__gQUBA"><a class="NewsList_link_news__pZ5s8" href="

In [53]:
# Map 함수를 활용해 뉴스 제목 리스트 만들기
list(map(lambda x: x.find('span', re.compile('NewsList_title')).text, news_tag_list))

['"상대 선수가 선 넘었다" 은메달 따고 눈물 \'펑펑\' 김민종에 쏟아진 위로·응원',
 '"왜 우리만 의심하나" 中선수 폭발…파리서도 미∙중 갈등, 무슨 일',
 '“상대 선수, 선넘었다”…은메달 따고 눈물 펑펑 김민종에 쏟아진 응원',
 "금메달 따고 동성 연인에 달려가 쪽…伊유도선수 '깜짝 세리머니'",
 '“올림픽 정신은 어디로?”...징계받은 조지아 유도 선수, 대체 어땠길래',
 "신유빈 '천적' 잡고 동메달 기회…'손목 부상' 하야타, 기권 가능성도",
 '금메달 딴 뒤 짝꿍에 청혼…한국 꺾은 중 배드민턴 혼복 선수에 환호',
 '\'한 발 0점\' 퇴장마저 극적... 김예지 "빅이벤트 선사해 실망 크셨을 것"',
 '“돌아가신 엄마 폰에 난 ‘금메달리스트’”...정나은, 銀 걸고 눈물',
 '“왜 나만 갖고 그래”…동메달 딴 中 미모의 수영선수 ‘울분’, 무슨일이']

In [54]:
# Map 함수를 활용해 뉴스 속성 리스트 만들기
list(map(lambda x: x.find('a', re.compile('NewsList_link')).get('href'), news_tag_list))

['/paris2024/article/011/0004375451',
 '/paris2024/article/025/0003377548',
 '/paris2024/article/009/0005344839',
 '/paris2024/article/025/0003377539',
 '/paris2024/article/009/0005344780',
 '/paris2024/article/008/0005072504',
 '/paris2024/article/028/0002701077',
 '/paris2024/article/023/0003850287',
 '/paris2024/article/023/0003850303',
 '/paris2024/article/009/0005344796']

In [55]:
# Selenium의 기능을 활용해 Xpath로 요소를 찾고, 버튼 클릭하기
news_more_button = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div[1]/div[2]/button')
news_more_button.click()

In [56]:
# Python 응용: 반복문을 활용해 버튼 클릭하기
import time

n = 0
for i in range(10):
    news_more_button.click()

    time.sleep(2) # 웹을 동작시킨 후, 정보를 불러올 수 있도록 충분히 시간을 기다려야 합니다.
    n += 1
    print(f'{n}회 클릭했습니다.')

1회 클릭했습니다.
2회 클릭했습니다.
3회 클릭했습니다.
4회 클릭했습니다.
5회 클릭했습니다.
6회 클릭했습니다.
7회 클릭했습니다.
8회 클릭했습니다.
9회 클릭했습니다.
10회 클릭했습니다.


In [57]:
# 다시 소스 코드 불러오기
html = driver.page_source # 웹을 동작시킨 뒤에는 다시 소스 코드를 불러와야 업데이트된 정보를 반영할 수 있습니다.
soup = BeautifulSoup(html, 'html.parser')

news_section = soup.find('div', re.compile('NewsList_comp_news_list'))
news_tag_list = news_section.find_all('li', re.compile('NewsList'))

news_title_list = list(map(lambda x: x.find('span', re.compile('NewsList_title')).text, news_tag_list))
news_link_list = list(map(lambda x: x.find('a', re.compile('NewsList_link')).get('href'), news_tag_list))

In [58]:
# 수집된 뉴스 개수 확인
print(f'뉴스 제목 수: {len(news_title_list)}, 뉴스 링크 수: {len(news_link_list)}')

뉴스 제목 수: 120, 뉴스 링크 수: 120


In [59]:
# Zip 함수를 활용해 뉴스 리스트 만들기
news_list = list(zip(news_title_list, news_link_list)); news_list

[('"상대 선수가 선 넘었다" 은메달 따고 눈물 \'펑펑\' 김민종에 쏟아진 위로·응원',
  '/paris2024/article/011/0004375451'),
 ('"왜 우리만 의심하나" 中선수 폭발…파리서도 미∙중 갈등, 무슨 일',
  '/paris2024/article/025/0003377548'),
 ('“상대 선수, 선넘었다”…은메달 따고 눈물 펑펑 김민종에 쏟아진 응원',
  '/paris2024/article/009/0005344839'),
 ("금메달 따고 동성 연인에 달려가 쪽…伊유도선수 '깜짝 세리머니'", '/paris2024/article/025/0003377539'),
 ('“올림픽 정신은 어디로?”...징계받은 조지아 유도 선수, 대체 어땠길래',
  '/paris2024/article/009/0005344780'),
 ("신유빈 '천적' 잡고 동메달 기회…'손목 부상' 하야타, 기권 가능성도",
  '/paris2024/article/008/0005072504'),
 ('금메달 딴 뒤 짝꿍에 청혼…한국 꺾은 중 배드민턴 혼복 선수에 환호',
  '/paris2024/article/028/0002701077'),
 ('\'한 발 0점\' 퇴장마저 극적... 김예지 "빅이벤트 선사해 실망 크셨을 것"',
  '/paris2024/article/023/0003850287'),
 ('“돌아가신 엄마 폰에 난 ‘금메달리스트’”...정나은, 銀 걸고 눈물',
  '/paris2024/article/023/0003850303'),
 ('“왜 나만 갖고 그래”…동메달 딴 中 미모의 수영선수 ‘울분’, 무슨일이',
  '/paris2024/article/009/0005344796'),
 ('“아직 은퇴할 생각 없다” 韓 올림픽 새 역사 도전장 내민 김우진 [2024 파리]',
  '/paris2024/article/241/0003370220'),
 ('"김민종, 승리 세리머니 비매너"…일본 언론 트집에 일본 네티즌도 콧방귀',
  '/paris

In [60]:
# 사용자 정의 함수를 활용해 튜플 형태로 데이터 불러오기

def get_news(news_tag):
    news_title = news_tag.find('span', re.compile('NewsList_title')).text
    news_link = news_tag.find('a', re.compile('NewsList_link')).get('href')

    return news_title, news_link

In [61]:
# Map 함수에 사용자 정의 함수 반영하여 뉴스 리스트 생성하기
news_list = list(map(get_news, news_tag_list)); news_list

[('"상대 선수가 선 넘었다" 은메달 따고 눈물 \'펑펑\' 김민종에 쏟아진 위로·응원',
  '/paris2024/article/011/0004375451'),
 ('"왜 우리만 의심하나" 中선수 폭발…파리서도 미∙중 갈등, 무슨 일',
  '/paris2024/article/025/0003377548'),
 ('“상대 선수, 선넘었다”…은메달 따고 눈물 펑펑 김민종에 쏟아진 응원',
  '/paris2024/article/009/0005344839'),
 ("금메달 따고 동성 연인에 달려가 쪽…伊유도선수 '깜짝 세리머니'", '/paris2024/article/025/0003377539'),
 ('“올림픽 정신은 어디로?”...징계받은 조지아 유도 선수, 대체 어땠길래',
  '/paris2024/article/009/0005344780'),
 ("신유빈 '천적' 잡고 동메달 기회…'손목 부상' 하야타, 기권 가능성도",
  '/paris2024/article/008/0005072504'),
 ('금메달 딴 뒤 짝꿍에 청혼…한국 꺾은 중 배드민턴 혼복 선수에 환호',
  '/paris2024/article/028/0002701077'),
 ('\'한 발 0점\' 퇴장마저 극적... 김예지 "빅이벤트 선사해 실망 크셨을 것"',
  '/paris2024/article/023/0003850287'),
 ('“돌아가신 엄마 폰에 난 ‘금메달리스트’”...정나은, 銀 걸고 눈물',
  '/paris2024/article/023/0003850303'),
 ('“왜 나만 갖고 그래”…동메달 딴 中 미모의 수영선수 ‘울분’, 무슨일이',
  '/paris2024/article/009/0005344796'),
 ('“아직 은퇴할 생각 없다” 韓 올림픽 새 역사 도전장 내민 김우진 [2024 파리]',
  '/paris2024/article/241/0003370220'),
 ('"김민종, 승리 세리머니 비매너"…일본 언론 트집에 일본 네티즌도 콧방귀',
  '/paris

In [65]:
# Pandas를 활용해 데이터 프레임 생성하기
import pandas as pd

df = pd.DataFrame(news_list, columns = ['title', 'url']); df

,title,url
0,"""상대 선수가 선 넘었다"" 은메달 따고 눈물 '펑펑' 김민종에 쏟아진 위로·응원",/paris2024/article/011/0004375451
1,"""왜 우리만 의심하나"" 中선수 폭발…파리서도 미∙중 갈등, 무슨 일",/paris2024/article/025/0003377548
2,"“상대 선수, 선넘었다”…은메달 따고 눈물 펑펑 김민종에 쏟아진 응원",/paris2024/article/009/0005344839
3,금메달 따고 동성 연인에 달려가 쪽…伊유도선수 '깜짝 세리머니',/paris2024/article/025/0003377539
4,"“올림픽 정신은 어디로?”...징계받은 조지아 유도 선수, 대체 어땠길래",/paris2024/article/009/0005344780
...,...,...
115,[올림픽]`금은 따 놓은 당상` 배드민턴 세계1위 안세영 출격,/paris2024/article/029/0002892405
116,"임시현 3관왕… 양궁 여제, ‘파리의 전설’ 됐다",/paris2024/article/023/0003850366
117,목에는 금메달 손에는 다이아반지…경기장서 청혼받은 中선수 ‘화제’,/paris2024/article/009/0005344804
118,김예지보다 쿨한 양지인?…‘대충 살다가’ 금메달 거머쥐었나[파리올림픽],/paris2024/article/018/0005804359


In [67]:
# Pandas의 내장 함수를 활용해 날짜 범위 계산하기
date_range = pd.date_range('2024-07-15', '2024-08-04').strftime('%Y%m%d'); date_range

Index(['20240715', '20240716', '20240717', '20240718', '20240719', '20240720',
       '20240721', '20240722', '20240723', '20240724', '20240725', '20240726',
       '20240727', '20240728', '20240729', '20240730', '20240731', '20240801',
       '20240802', '20240803', '20240804'],
      dtype='object')

In [76]:
# 반복문을 활용해 날짜별로 데이터 수집하기
from tqdm.auto import tqdm

total_news_list = []
for date in tqdm(date_range):
    url = f'https://m.sports.naver.com/paris2024/news?date={date}&sort=popular&isPhoto=N'

    driver.get(url)
    time.sleep(2)

    news_more_button = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div[1]/div[2]/button')

    n = 0
    for i in range(5):
        try:
            news_more_button.click()
            time.sleep(2)
            n += 1
            print(f'{date}: {n}회 클릭했습니다.')
        except:
            next

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    news_section = soup.find('div', re.compile('NewsList_comp_news_list'))
    news_tag_list = news_section.find_all('li', re.compile('NewsList'))

    news_title_list = list(map(lambda x: x.find('span', re.compile('NewsList_title')).text, news_tag_list))
    news_link_list = list(map(lambda x: x.find('a', re.compile('NewsList_link')).get('href'), news_tag_list))
    date_list = [date] * len(news_link_list)

    daily_news_list = list(zip(news_title_list, news_link_list, date_list))
    total_news_list.extend(daily_news_list)

  0%|          | 0/21 [00:00<?, ?it/s]

20240715: 1회 클릭했습니다.
20240716: 1회 클릭했습니다.
20240716: 2회 클릭했습니다.
20240717: 1회 클릭했습니다.
20240718: 1회 클릭했습니다.
20240718: 2회 클릭했습니다.
20240718: 3회 클릭했습니다.
20240718: 4회 클릭했습니다.
20240718: 5회 클릭했습니다.
20240719: 1회 클릭했습니다.
20240719: 2회 클릭했습니다.
20240719: 3회 클릭했습니다.
20240719: 4회 클릭했습니다.
20240719: 5회 클릭했습니다.
20240720: 1회 클릭했습니다.
20240720: 2회 클릭했습니다.
20240720: 3회 클릭했습니다.
20240720: 4회 클릭했습니다.
20240720: 5회 클릭했습니다.
20240721: 1회 클릭했습니다.
20240721: 2회 클릭했습니다.
20240721: 3회 클릭했습니다.
20240721: 4회 클릭했습니다.
20240721: 5회 클릭했습니다.
20240722: 1회 클릭했습니다.
20240722: 2회 클릭했습니다.
20240722: 3회 클릭했습니다.
20240722: 4회 클릭했습니다.
20240722: 5회 클릭했습니다.
20240723: 1회 클릭했습니다.
20240723: 2회 클릭했습니다.
20240723: 3회 클릭했습니다.
20240723: 4회 클릭했습니다.
20240723: 5회 클릭했습니다.
20240724: 1회 클릭했습니다.
20240724: 2회 클릭했습니다.
20240724: 3회 클릭했습니다.
20240724: 4회 클릭했습니다.
20240724: 5회 클릭했습니다.
20240725: 1회 클릭했습니다.
20240725: 2회 클릭했습니다.
20240725: 3회 클릭했습니다.
20240725: 4회 클릭했습니다.
20240725: 5회 클릭했습니다.
20240726: 1회 클릭했습니다.
20240726: 2회 클릭했습니다.
20240726: 3회 클릭했습니다.
20240726: 4회 

In [79]:
# 수집한 데이터로 최종 데이터 프레임 생성하기
df = pd.DataFrame(total_news_list, columns = ['title', 'url', 'date']); df

,title,url,date
0,"BTS 진, 수천 명 '아미' 환호 속 파리올림픽 성화 봉송",/paris2024/article/001/0014806251,20240715
1,"'우상혁 라이벌' 바르심, 올림픽 앞둔 마지막 실전 2ｍ31로 우승",/paris2024/article/001/0014806332,20240715
2,"[센터연예] BTS 진, 아미들 환호 속 파리 올림픽 성화 봉송",/paris2024/article/422/0000670725,20240715
3,"BTS 진, 수천명 아미 환호 속 파리 올림픽 성화 봉송 ‘성공적’…“너무 영광”",/paris2024/article/144/0000975442,20240715
4,"박인비, 뱃속 둘째와 IOC 선수위원 도전 ""엄마의 힘 보여줄게요""",/paris2024/article/015/0005009774,20240715
...,...,...,...
1123,"신유빈 매너에 반한 일본…""패자의 품격, 앞으로도 응원할래"" [올림픽]",/paris2024/article/421/0007708660,20240804
1124,왜 안바울은 7kg이 더 나가는 선수와 싸웠나…한국 유도 감격의 동메달 획득 [올림...,/paris2024/article/477/0000505555,20240804
1125,세계 3위 상대로 끝까지 투혼... 女핸드볼 “태극기 있어 힘냈다”,/paris2024/article/005/0001715436,20240804
1126,‘천적’ 중국 천위페이 ‘충격’의 8강 탈락…28년만 금메달 ‘도전’ 안세영에게도 ...,/paris2024/article/468/0001082787,20240804


In [80]:
# CSV 데이터 저장하기

import os

path = '/content/drive/MyDrive/[deep daiv.] 딥러닝 입문 2024/2강. 파이썬 응용 및 크롤링'
file_name = f'파리올림픽_네이버_뉴스_{date_range[0]}_{date_range[-1]}.csv'

df.to_csv(os.path.join(path, file_name), index = False)

# [복습 과제1] 2024년 8월 '양궁' 관련 뉴스 본문 수집하기

1. 앞서 제시한 코드를 활용하여, '양궁' 분야 URL을 가지고 옵니다. ('PARIS NOW > '뉴스' > '양궁'을 클릭합니다.)
2. 날짜 범위를 2024년 8월로 조정합니다. 수집해야 할 일별 뉴스는 30개입니다.
3. '제목', '링크'에 추가로 '조회수'를 파싱해 리스트로 생성합니다.
4. `discipline_list` 리스트를 생성하고, 뉴스 길이만큼 'ARC'로 값을 채워 데이터 프레임에 컬럼을 하나 더 추가합니다.
5. 최종적으로 '제목', 'URL', '조회수', '날짜', '종목' 5개의 컬럼을 생성한 데이터프레임을 만들어 CSV 파일을 추출합니다. Slack 채널에는 데이터프레임 이미지를 캡처해 댓글로 업로드합니다.

In [93]:
# 필요한 라이브러리는 미리 불러옵니다.
date_range = pd.date_range('2024-08-01', '2024-08-04').strftime('%Y%m%d')

total_news_list = []
for date in tqdm(date_range):
    url = f'https://m.sports.naver.com/paris2024/news?disciplineId=ARC&date={date}&sort=popular&isPhoto=N'

    driver.get(url)
    time.sleep(2)

    news_more_button = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div[1]/div[2]/button')

    n = 0
    for i in range(3):
        try:
            news_more_button.click()
            time.sleep(2)
            n += 1
            print(f'{date}: {n}회 클릭했습니다.')
        except:
            next

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    news_section = soup.find('div', re.compile('NewsList_comp_news_list'))
    news_tag_list = news_section.find_all('li', re.compile('NewsList'))

    news_title_list = list(map(lambda x: x.find('span', re.compile('NewsList_title')).text, news_tag_list))
    news_link_list = list(map(lambda x: x.find('a', re.compile('NewsList_link')).get('href'), news_tag_list))
    news_hits_list = list(map(lambda x: x.find('span', re.compile('NewsList_data')).text, news_tag_list))

    date_list = [date] * len(news_link_list)
    discipline_list = ['ARC'] * len(news_link_list)

    daily_news_list = list(zip(news_title_list, news_link_list, date_list, news_hits_list, discipline_list))
    total_news_list.extend(daily_news_list)

  0%|          | 0/4 [00:00<?, ?it/s]

20240801: 1회 클릭했습니다.
20240801: 2회 클릭했습니다.
20240801: 3회 클릭했습니다.
20240802: 1회 클릭했습니다.
20240802: 2회 클릭했습니다.
20240802: 3회 클릭했습니다.
20240803: 1회 클릭했습니다.
20240803: 2회 클릭했습니다.
20240803: 3회 클릭했습니다.
20240804: 1회 클릭했습니다.
20240804: 2회 클릭했습니다.
20240804: 3회 클릭했습니다.


In [94]:
df = pd.DataFrame(total_news_list, columns = ['title', 'url', 'date', 'hits', 'discipline']); df

,title,url,date,hits,discipline
0,"""'미스터 오' 덕분""…단체전 첫 메달 딴 프랑스 양궁, 한국 감독 '찬사'",/paris2024/article/008/0005071562,20240801,"조회수 189,245",ARC
1,양궁 김우진 옆 1점 쏜 차드 선수 “고마워요 한국”,/paris2024/article/005/0001715059,20240801,"조회수 116,924",ARC
2,"""김제덕 보려고 안 잤는데""…양궁 개인전 우천으로 연기",/paris2024/article/025/0003377141,20240801,"조회수 95,213",ARC
3,프랑스 양궁 선수가 꼽는 ‘한국 양궁 신화’ 비결은? [특파원 리포트],/paris2024/article/056/0011773197,20240801,"조회수 85,323",ARC
4,‘양궁 최강’ 韓엔 졌지만… 中-佛 은메달도 한국인이 이끌었다,/paris2024/article/020/0003579663,20240801,"조회수 59,379",ARC
...,...,...,...,...,...
155,"[올림픽] 임시현 ""항저우·파리서 다 3관왕 할 확률은?…바늘구멍 뚫었죠""",/paris2024/article/055/0001178383,20240804,"조회수 31,575",ARC
156,"""도쿄올림픽 챔피언을 꺾습니다!"" ""준결승 이우석과 맞붙습니다""",/paris2024/article/214/0001365739,20240804,"조회수 31,480",ARC
157,"'맏언니'는 달랐다…정의선 회장, 전훈영에 감사 표한 이유는? [2024 파리올림픽]",/paris2024/article/015/0005017773,20240804,"조회수 30,551",ARC
158,양궁 '3관왕' 임시현 세리머니의 비밀,/paris2024/article/057/0001833746,20240804,"조회수 30,269",ARC


# [복습 과제2] 뉴스 본문 수집하기

1. 앞서 실습에서 진행한 데이터프레임을 불러와 학습한 개념을 응용해 각 URL로 접속하여 뉴스 본문을 수집합니다.
2. 제목, (입력) 날짜, 본문, 언론사 정보, URL을 하나의 데이터프레임으로 생성해 추출합니다.
3. 최종적으로 '제목', '날짜', '본문', '언론사', 'url' 5개의 컬럼을 생성한 데이터프레임을 만들어 CSV 파일을 추출합니다. Slack 채널에는 데이터프레임 이미지를 캡처해 댓글로 업로드합니다.

In [134]:
url_list = df['url']
data_list = []

for url in url_list[:5]: # 예시를 위해 5개만 수집했습니다.
    full_url = 'https://m.sports.naver.com' + url
    driver.get(full_url)
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find('h2').text
    date = soup.find('em', re.compile('NewsEndMain_date')).text
    content = soup.find('div','_article_content').text
    press = soup.find('a', re.compile('NewsEndMain_article_head_press_logo')).img.get('alt')

    print(title, date, content, press, url, sep = '\n')
    print('-' * 10)
    data = title, date, content, press, url

    data_list.append(data)

"'미스터 오' 덕분"…단체전 첫 메달 딴 프랑스 양궁, 한국 감독 '찬사'
2024.08.01. 오전 9:21
대한민국 양궁대표팀 김우진, 이우석, 김제덕이 지난 30일(한국시간) 프랑스 파리 앵발리드에서 열린 2024 파리올림픽 양궁 남자 단체전 시상식에서 금메달을 목에 건 후 프랑스(은메달), 터키(동메달) 선수들과 기념촬영을 하고 있다. /사진=뉴스1 프랑스가 남자 양궁 단체전에서 사상 최초로 메달을 획득한 것을 두고 외신은 물론, 프랑스 현지에서도 한국 감독 덕분이라는 찬사가 쏟아지고 있다. 1일 프랑스 일간신문인 르몽드는 지난달 28일 프랑스 양궁협회가 올해 2024 파리올림픽을 앞두고 최소한 하나의 메달 획득을 목표로 전례 없는 지원을 퍼부으면서 한국인 감독을 선임해 효과를 봤다고 보도했다. 프랑스 남자 단체전 은메달을 이끈 한국인 감독은 '미스터 오(Mr. Oh)'로 알려진 오선택 감독이다. 르몽드는 오 감독에 대해 업계에서 잘 알려진 사람이라고 보도했다. 그가 2000년과 2012년 두 차례에 걸쳐 한국 국가대표팀 감독을 맡았고 2000년 시드니(호주)의 윤미진, 2012년 런던의 오진혁, 2016년의 장혜진 등 여러 선수들을 올림픽 시상대 가장 높은 곳에 올려놨다고 소개했다. 한국은 양궁의 '기준점'이고, 1972년 뮌헨올림픽에서 양궁이 올림픽 정식종목으로 복귀한 후 한국 대표팀이 전체 45개 종목 중 27개 종목에서 우승을 차지했다고도 보도했다. 프랑스 남자 양궁 대표팀 리더인 장 샤를 발라동(Jean-Charles Valladont)이 "한국은 우수한 양궁 선수들이 너무 많다"면서 "테디 리네르(프랑스의 세계 최강 유도선수)와 같은 스타들이 많고 완전히 다른 세상이다"고 인터뷰한 내용도 실었다. 프랑스 세바스티앙 플루트 양궁 센터에서 'Mr. Oh'로 알려진 한국의 오선택 감독올림픽 시뮬레이션을 하고 있다./사진=르몽드 캡처그러면서 한국 양궁의 성공 역사는 1988년 서울올림픽 유치에 뿌리를 두고 있다고도 분석했다. 한국 정부가 1988년

In [136]:
df = pd.DataFrame(data_list, columns = ['title', 'date', 'content', 'press' , 'url']); df

,title,date,content,press,url
0,"""'미스터 오' 덕분""…단체전 첫 메달 딴 프랑스 양궁, 한국 감독 '찬사'",2024.08.01. 오전 9:21,"대한민국 양궁대표팀 김우진, 이우석, 김제덕이 지난 30일(한국시간) 프랑스 파리 ...",머니투데이,/paris2024/article/008/0005071562
1,양궁 김우진 옆 1점 쏜 차드 선수 “고마워요 한국”,2024.08.01. 오후 4:51,2024 파리 올림픽 남자 양궁 개인전 64강에서 한국 남자 양궁 국가대표 김우진과...,국민일보,/paris2024/article/005/0001715059
2,"""김제덕 보려고 안 잤는데""…양궁 개인전 우천으로 연기",2024.08.01. 오전 7:04,김제덕이 지난 29일(현지시간) 프랑스 파리 레쟁발리드 양궁 경기장에서 열린 ...,중앙일보,/paris2024/article/025/0003377141
3,프랑스 양궁 선수가 꼽는 ‘한국 양궁 신화’ 비결은? [특파원 리포트],2024.08.01. 오전 8:00,한국 양궁 대표팀이 이번 파리올림픽 여자 단체전과 남자 단체전에서 잇따라 우승을 거...,KBS,/paris2024/article/056/0011773197
4,‘양궁 최강’ 韓엔 졌지만… 中-佛 은메달도 한국인이 이끌었다,2024.08.01. 오전 3:02,[PARiS 2024]출전한 해외 8개국 감독이 ‘코리안’… 韓 지도자 진출에 ‘양...,동아일보,/paris2024/article/020/0003579663
